### Table of Content
- Chapter  2 : Preliminary Knowledge
- Chapter  3 : Linear Neural Network
- Chapter  4 : Classification
- Chapter  5 : Multilayer Perceptrons
- Chapter  6 : Beginner Guide
- Chapter  7 : CNN
- Chapter  8 : Modern CNN
- Chapter  9 : RNN
- Chapter 10 : Modern RNN
- Chapter 11 : Attention and Transformer
- Chapter 12 : Optimization
- Chapter 13 : Computation Performance
- Chapter 14 : Computer Vision
- Chapter 15 : NLP (pre-tranining)
- Chapter 16 : NLP (applications)
- Chapter 17 : Reinforcement Learning
- Chapter 18 : Gaussian Processes
- Chapter 19 : Hyperparameter Optimization
- Chapter 20 : Generative Adversarial Networks
- Chapter 21 : Recommender Systems
  
- Appendenix : Engineer and Hardware Optimization RoadMap
- Appendenix : Gradient Check Techniques.
- Useful Jupyter Lab Tips:
  - %whos for variables in memory

### Chapter 2 : Preliminary Knowledge
- 数据操作
  - 广播机制（两个数据分别复制扩充到同样的尺寸）
  - 节省内存（使用X[:] = \<expression\>或X+=\<expression\>来避免重新分配）
- 数据预处理
- 线性代数 
  - 转置.T 范数norm
  - 非降维求和 (keepdims=True)，累积和cumsum
  - torch.dot只支持向量，矩阵和向量间用mv，矩阵之间用mm
- 微积分
  - 设T是梯度算符，T(Ax) = A.T, T(x.T·A) = A, T(x.T A x) = (A + A.T)x
- 自动微分
  - 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
  - 自动微分必须是标量，非标量的话要么转成标量，要么指定输出形状
  - 分离操作
- 概率论
- 查阅文档、API的指导
  - dir查看可以调用的函数和类

### Chapter 3 : Linear Neural Network
- Minibatch stochastic gradient descent (小批量随机梯度下降)
- 一般的训练过程
  - model.forward() 与 y_hat 做差，然后反向传播，优化器根据导数去更新参数
- Machine Learning Concept
  - lasso regression: l1 norm; ridge regression: l2 norm;

## Chapter 4 : Classification
- softmax:
  $y_i = \frac{\exp(o_i)}{\sum_j \exp(o_j)}$, often minus max(oj) to get numerical stable
- Information theory
  - cross-entropy loss：$l(y, \hat y) = - \sum y_i * \log(\hat y_i)$
  - amount of information $\log{\frac{1}{P(j)}} = - \log{P(j)}$ 
  - entorpy $H[P] = \sum -P(j) \log{P(j)}$
  - cross-entorpy $H(P, Q) = \sum -P(j) \log{Q(j)}, ~ P=Q \rightarrow H(P, Q) = H(P, P) = H(P)$. In pytorch, F.cross_entropy will do the softmax for you.
- Image Classification Rules:
  - image stored in (channel, height, weight) manner.
- Distrubution shift:
  - Covariate Shift (feature shift): $p(x) \neq q(x), p(y|x) = q(y|x)$
    - For example: p(x) and q(x) are features of oral and urban house, y is the price, we assume the feature and label relation is the same
    - Method: weighted by $$\beta(x) = p(x) / q(x) \rightarrow \int\int l(f(x), y)p(y|x)p(x)dxdy = \int\int l(f(x), y)q(y|x)q(x) \frac{p(x)}{q(x)}dxdy \rightarrow \sum_i \beta_i l(f(x_i), y_i)$$ $\beta$ can be obtained with logistic regression.
  - Label Shift, $p(y) \neq q(y), p(x|y) = q(x|y)$, the same method $\beta(y) = p(y) / q(y)$, but now $q(y)$ is hard to get, we need compute a confusion matrix on the val data then use the model to pridcit the distrubution of the $q(y)$
  - Concept Shift (the concept of the label)

## Chapter 5 : Multilayer Perceptrons
- Activation Function: relu, sigmoid, tanh ($\frac{1 - \exp(-2x)}{1 + \exp(-2x)}$)
- Numerical stability: vanish and explode are common
  - Symmetry: linear layer and conv (with no share weight) layer are symmetric so we can not tell apart from different weight and try to explain it (for example 2 hidden unit with same initial value, they will update the same way), so we need to **Bread the Symmetry** (like using a dropout)
  - Xavier initilization: get from distrubution of zero mean and variance $\sigma = \sqrt{2 / (n_{in} + n_{out})}$
  - Dropout, shared param...
- (Rolnick et al., 2017) has revealed that in the setting of label noise, neural networks tend to fit cleanly labeled data **first** and only subsequently to interpolate the mislabeled data.
  - so we can use early stop once error on val is minimal or the patience hit. usually combined with regularization.
- Dropout:
  - $h^{'} = \left \{ 
  \begin{array}{lll}
  & 0, p \\
  & \frac{h}{1-p}, 1-p
  \end{array} 
  \right .$, now $E[h^{'}] = E[h]$
  - We do not use dropout in test, except we want to know the uncertainty of the model output (by comparing different dropout)
  - Use lower p in lower layer (to get lower feature), higher p in higher layer

## Chapter 6: Beginner Guide
- Tied layer: gradient will add up along different chain
- Custom initialization: `apply` method
- I/O
  - save tensor: `torch.save(x:Uinon[List[tensor], Dict], name:str)` and load
  - save model: the same, just input dict of the net (`net.state_dict()`) then `net.load_state_dict(torch.load(name))`
- GPU
  - operation between tensors must in the same GPU
  - print or transform to numpy will copy to memory, and even worse wait the python **GIL** (`Global Interpreter Lock`, make sure at the same time only one thread can execute the python bytecode)

## Chapter 7 : CNN
1. **Invariance**: translation equivariance, locality -> The earliest layers should respond similarly to the same patch and focus on local regions.
2. **Convolution**: math is $(f * g)(i, j) = \sum_a \sum_b f(a, b)  g(i - a, j - b)$, remind that **cross-correlation** is $(f * g)(i, j) = \sum_a \sum_b f(a, b)  g(i + a, j + b)$
   - The difference is not important as we will learn the kernel, `k_conv_learned = k_corr_learned.T`, or `conv(X, k_conv_learned) = corr(X, k_corr_learned)`
3. **Receptive Field**： for any element (tensors on the conv layer) x, all the elements that may effect x in the previous layers in the forward population.
4. **Padding, Stride**: $\lfloor (n_h - k_h + p_h + s_h) / s_h \rfloor \times \lfloor (n_w - k_w + p_w + s_w) / s_w \rfloor$, often `p_h = k_h - 1`, the same for `p_w`. `p_h = p_h_upper + p_h_lower`
5. **Channel**:
   - multi in $c_i$ -> kernel must also have the same channels ($c_i \times k_h \times k_w$), then add them up.
   - multi out $c_o$ -> kernel with $c_o \times c_i \times k_h \times k_w$, get $c_o$ output channels.
6. use `torch.stack` to stack tensors
7. **Pooling**: mitigating the sensitivity of convolutional layers to location and of spatially downsampling representations.

## Chapter 8 : Modern CNN
1. **AlexNet**: first deep conv successful, using dropout, Relu, polling
2. **VGG**: multiple 3 * 3 conv layers (two 3 * 3 conv touch 5 * 5 input as a 5 * 5 conv, but 2 * 3 * 3  = 18 < 25 = 5 * 5)
3. **NiN**: to handle 2 problem (1. much ram for the MLP at the end; 2. can not add MLP between the conv to increase the degree of nonlinearity as it will destroy the spatial information)
   - use 1 * 1 conv layer to add local nonlinearities across the channel activations
   - use global average pooling to integrate across all locations in the last representation layer. (must combine with added nonlinearities)
4. **GoogleNet**: Inception layer, parallel conv multi scales, and then concate them
5. **Batch Normalization**:
   - $BN(\mathbf x) = \mathbf{\gamma} \bigodot \frac{\mathbf x - \mathbf{\mu_B}}{\sigma^2_B} + \mathbf \beta$, $\mathbf{\mu_B} = \frac{1}{|B|}\sum_{x \in B} \mathbf x$,
     $\sigma^2_B = \frac{1}{|B|} \sum_{x \in B} (x - \mathbf{\mu_B})^2 + \epsilon$
   - On linear layer [N, D] it will get across D (different features in D will not do calculations), on conv layer [N, C, H, W] it will across C (save the difference between channels)
     - For example, [N, C, H, W] shape input x, for x[N, 0, H, W], get it's mean mu and std and do (x[N, 0, H, W] - mu) / std, here mu and std are scalar.
   - At the testing stage, we will use the global (whole) data mean and varience, instead of minibatch mean and varience. Just like dropout.
   - So BN also serves as a noise introducer! (minibatch information != true mean and var) Teye et al. (2018) and Luo et al. (2018).
   - So it best works for batch size of 50 ~ 100, higher the noise is small, lower it is too high.
   - Moving global mean and var: when testing, no minibatch, so we use a global one that is stored during training.
     - It is a kind of exp weighted mean, closest batch has higer weight
     - $\mu_m = \mu_m * (1 - \tau) + \mu * \tau, \Sigma_m = \Sigma_m * (1 - \tau) + \Sigma * \tau$, $\tau$ is called momentum term.
6. **Layer Normalization**: often used in NLP
   - For features like [N, A, B] it will save difference between N, A and B are typically seq_len, hidden_size.
7. **ResNet**: residual block, pass x as one of the branch before a activation function (for the original paper, and later it is changed to BN -> AC -> Conv)
   - To get the passed x has the correct shape to add up, we can use 1 * 1 conv if it is needed
   - **Idea**: nested-function class, shallower net (like ResNet-20) is subclass of depper net (like ResNet-50). Because in ResNet-50 if the layers after 20th layer are f(x) = x, then it is the same as RestNet-20! So we can make sure f' (the best we can get in ResNet-50 for certain data) will be better than f (ResNet-20 on the same data) or at least the same.
   - <figure style="text-align: center;">
      <img alt="Residul Block" src="https://d2l.ai/_images/resnet-block.svg" style="background-color: white; display: inline-block;">
      <figcaption> Residul Block </figcaption>
    </figure>
    
   - **ResNeXt**: use g groups of 3 * 3 conv layers between two 1 * 1 conv of channel $b$ and $c_o$, so $\mathcal O(c_i c_o) \rightarrow \mathcal O(g ~ c_i / g ~ c_o / g) = \mathcal O(c_ic_o/g)$
     - This is a **Bottleneck** arch if $b < c_i$
       </br>
   - <figure style="text-align: center;">
      <img alt="ResNeXt Block" src="https://d2l.ai/_images/resnext-block.svg" style="background-color: white; display: inline-block;">
      <figcaption> ResNeXt Block </figcaption>
    </figure>
    
8. **DenseNet**: instead of plus x, we concatenate x repeatedly.
   - For example (\<channel\> indicates the channel): x\<c_1\> -> f_1(x)\<c_2\> end up with [x, f_1(x)]\<c_1 + c_2\> -> f_2([x, f_1(x)])\<c_3\> end up with [x, f_1(x), f_2([x, f_1(x)])]\<c_1 + c_2 + c_3\>
   - Too many of this layer will cause the dimeansion too big, so we need some layer to reduce it. **Translation** layer use 1 * 1 conv to reduce channel and avgpool to half the H and W.
9. **RegNet**:
   - AnyNet: network with **stem** -> **body** -> **head**.
   - Distrubution of net: $F(e,Z)=∑_{i=1}^{n}1(e_i<e)$, use this empirical CDF to approximate $F(e, p)$, $p$ is the net arch distrubution. $Z$ is a sample of net sample from $p$, if $F(e, Z_1) < F(e, Z_2)$ then we say $Z_1$ is better, it's parameters are better.
   - So for RegNet, they find that we should use same k (k = 1, no bottlenet, is best, says in paper) and g for the ResNeXt blocks with no harm, and increase the network depth d and weight c along the stage. And keep the c change linearly with $c_j = c_o + c_aj$ with slope $c_a$
   - neural architecture search (NAS) : with certain search space, use RL (NASNet), evolution alg (AmoebaNet), gradient based (DARTS) or shared weight (ENAS) to get the model. But it takes to much computation resource.
   - <figure style="text-align: center;">
      <img src="https://d2l.ai/_images/anynet.svg" style="background-color: white; display: inline-block;">
      <figcaption> AnyNet Structure (Search Space) </figcaption>
    </figure>
   </br>

## Chapter 9 : RNN
- Two form of sequence to sequence task:
  - **aligned**: input at certain time step aligns with corrsponding output, like tagging (fight -> verb)
  - **unaligned**: no step-to-step correspondence, like maching translation
- **Autoregressive** model: regress value based on previous value
  - latent autoregressive models (since $h_t$ is never observed): estimate $P(x_t | x_{t-1} \dots x_1)$ with $\hat x_t = P(x_t | h_t)$ and $h_t = g(h_{t-1}, x_{t-1})$
- **Sequence Model**: to get joint probablity of a sequence $p(x_1, \dots, x_T)$, we change it to a form like autoregressive one: $p(x_1) \prod_{t=2}^T p(x_t|x_{t-1}, \dots, x_1)$
  - **Markov Condition**: if we can make the condition above into $x_{t-1}, \dots, x_{t-\tau}$ without any loss, aka the future is conditionally independent of the past, given the recent history, then the sequence satisfies a Markov condition. And it is $\tau^{th}$-order Markov model.
- Zipf’s law: the frequency of words will decrease exponentially, n-grams too (with smaller slope).
  - So use word frequency to construct the probility is not good, for example. $\hat p(learning|deep) = n(deep, learning) / n(deep)$, $n(deep, learning)$ will be very small compared to denominator. We can use so called **Laplace Smooth** but that will not help too much.
- **Perplexity**: (how confusion it is), given a true test data, the cross-entropy is $J = \frac{1}{n} \sum_{t=1}^n -\log P(x_t | x_{t-1}, \dots, x_1)$, and the perplexity is $\exp(J)$.
- Partioning the sequence: for a $T$ token indices sequence, we add some randomness, discard first $d \in U(0, n]$ tokens and partion the rest into $m = \lfloor (T-d) / n \rfloor$ group. For a sequence $x_t$ the target sequence is shifted by one token $x_{t+1}$.
---
- **RNN**: for a vocab with size $|\mathcal V|$, the model parameters should go up to $|\mathcal V|^n$, $n$ is the sequence length.So we $P(x_t | x_{t-1} \dots x_1) \approx P(x_t | h_{t-1})$，$h$ is a **hidden state**, it varies at different time step and contains information of previous time steps. Hidden layer, on the other hand, is a structure, it dose not change in forward calculation.
  - recurrent: $H_t = \phi (X_tW_{th} + H_{t-1}W_{hh} + b_h)$, output is $O_t = H_tW_{tq} + b_q$.
  - <figure style="text-align: center;">
      <img alt="ResNeXt Block" src="https://d2l.ai/_images/rnn.svg" style="background-color: white; display: inline-block;">
      <figcaption> ResNeXt Block </figcaption>
    </figure>
  - clip the gradient: $g = \min(1, \frac{\theta}{|| g ||}) g$, it is a hack but useful.
  - **Warm-up**: When predicting, we can first feed a prefix (now called prompt I think), just iter the prefix into the network without generating output until we need to predict.
- For RNN: the input shape is (sequence_length, batch_size, feature_size), first is time_step, third is one-hot dim or word2vec dim.
- **Backpropagation through time**
  - <figure style="text-align: center;">
      <img alt="ResNeXt Block" src="https://d2l.ai/_images/rnn-bptt.svg" style="background-color: white; display: inline-block;">
      <figcaption> Computation graph of RNN </figcaption>
    </figure>
  - How to reduce gradient explosion or vanishing: truncate the gradient propagete at certain time step.
  - In the img above: $$\begin{array}{lll}\frac{\partial L}{\partial h_T} = W_{qh}^{\intercal} \frac{\partial L}{\partial o_T} \\\frac{\partial L}{\partial h_t} = \sum_{i=t}^T (W_{hh}^{\intercal})^{T-i} W_{qh}^{\intercal} \frac{\partial L}{\partial o_{T+t-i}} \\ \frac{\partial L}{\partial W_{hx}} = \sum_{t=1}^T \frac{\partial L}{\partial h_t} x_t^{\intercal} \\ \frac{\partial L}{\partial W_{hh}} = \sum_{t=1}^T \frac{\partial L}{\partial h_t} h_{t-1}^{\intercal}\end{array}$$

## Chapter 10 : Modern RNN
- **LSTM**
  - The structure is :
  - <figure style="text-align: center;">
      <img alt="LSTM Arch" src="https://zh.d2l.ai/_images/lstm-3.svg" style="background-color: white; display: inline-block;">
      <figcaption> LSTM Arch </figcaption>
    </figure>
- **GRU**
  - <figure style="text-align: center;">
      <img alt="GRU Arch" src="https://d2l.ai/_images/gru-3.svg" style="background-color: white; display: inline-block;">
      <figcaption> GRU Arch </figcaption>
    </figure>
  - Reset gates help capture short-term dependencies in sequences.
  - Update gates help capture long-term dependencies in sequences.
- **Deep RNN**
  - <figure style="text-align: center;">
      <img alt="Deep RNN" src="https://d2l.ai/_images/deep-rnn.svg" style="background-color: white; display: inline-block;">
      <figcaption> Deep RNN </figcaption>
    </figure>
  - In deep rnn, the output is the last layer of the hidden state with every timestep, and state is the last time step hidden state with all layer of rnn.
- **Bidirection RNN**, it is slow and gradient chain is long
  - $P(x_1,\ldots,x_T,h_1,\ldots,h_T)=\prod_{t=1}^TP(h_t\mid h_{t-1})P(x_t\mid h_t),\mathrm{~where~}P(h_1\mid h_0)=P(h_1)$, it is a hidden markov model. We can use dynamic programming method compute is from start to end, also from end to start. Just how B-RNN is capable of.
  - <figure style="text-align: center;">
      <img alt="Bidirection RNN" src="https://zh.d2l.ai/_images/birnn.svg" style="background-color: white; display: inline-block;">
      <figcaption> B-RNN </figcaption>
    </figure>
  - And we just need to concatenate these two H.
- **Machine translation**
  - non-breaking space, some space should not split to new line, like Mr. Smith.
  - Teacher Forcing : all the input will be pad with \<pad\>, source token no special treat, decoder input (target seq use as input) will start with \<bos\>, and label is shift by 1 (no \<bos\> at the begining).
  - **Important**: when use teacher forcing, the truth target is feed to the decoder. This will make the traning faster and stable, but it will make training and predicting different (because when predicting we do not have truth target label, we have to repeatedly predict). We can make them the same, but the tranning will be harder.
- **Sequence to Sequence**
  - We use this Encoder - Decoder Arch to get varied length input and varied length output.
  - We do not use one-hot, instead we use nn.Embed layer, which will take token i, and return ith row of the matrix of this embeding layer.
  - From the encoder, we get the hidden states, and use a funcion $c = q(h_1, \cdots, h_T)$, for example, just use the $h_T$. And in the decoder, we concatenate this with the target embed output, and feed to rnn.
  - When calculating the loss, we should not take \<pad\> into acount. So we need to musk the loss with the tokens.
  - <figure style="text-align: center;">
      <img alt="Encoder Decoder" src="https://d2l.ai/_images/seq2seq-details.svg" style="background-color: white; display: inline-block;">
      <figcaption> Encoder Decoder </figcaption>
    </figure>
  - Bilingual Evaluation Understudy, BLEU evaluates whether this n-gram in the predicted sequence appears in the target sequence. For example, target sequence ABCDEF, predict sequence ABBCD, $p_1 = 4/5$, we have ABCD in the target sequence, $p_2 = 3 / 4$, we have AB, BC, CD. So we get BLEU as $\exp\left(\min\left(0,1-\frac{\mathrm{len}_{\mathrm{label}}}{\mathrm{len}_{\mathrm{pred}}}\right)\right)\prod_{n=1}^kp_n^{1/2^n}$, higher n will have higher weight, small length of predict length takes lower.
- **Beam Search**
  - Before this section, we use greedy search to get prediction, use argmax on the prediction vector : $y_{t^{\prime}}=\underset{y\in\mathcal{Y}}{\operatorname*{\operatorname*{argmax}}}P(y\mid y_1,\ldots,y_{t^{\prime}-1},\mathbf{c})$, where $\mathcal Y$ is the vacab. Once our model outputs “<eos>” (or we reach the maximum length $T'$) the output sequence is completed.
  - However, use the most likely tokens is not the same with the most likely sequence : $\prod_{t^{\prime}=1}^{T^{\prime}}P(y_{t^{\prime}}\mid y_1,\ldots,y_{t^{\prime}-1},\mathbf{c})$. For example, in this figure below, ACB will have this probability of 0.5 * 0.3 * 0.6 = 0.09. On the other hand, greedy search choose ABC which is 0.5 * 0.4 * 0.4 = 0.08, it is lower, not optimal!
  - <figure style="text-align: center;">
      <img alt="Max sequence" src="https://d2l.ai/_images/s2s-prob2.svg" style="background-color: white; display: inline-block;"><img alt="Max token" src="https://d2l.ai/_images/s2s-prob1.svg" style="background-color: white; display: inline-block;">
      <figcaption> Max sequence (left), Max token (right) </figcaption>
    </figure>
  - If we want the optimal one, we need to do exhaustive search, search all possible sequence, it is not possible!
  - The most straightforward type of beam search is keep k candidates. In time step 2, we get $P ( A, y_{2} \mid\mathbf{c} )=P ( A \mid\mathbf{c} ) P ( y_{2} \mid A, \mathbf{c} )$ for the top, and $P ( C, y_{2} \mid\mathbf{c} )=P ( C \mid\mathbf{c} ) P ( y_{2} \mid C, \mathbf{c} ) $ for the bottom, then choose most 2 from them. And then choose sequence that maximize $$\frac{1} {L^{\alpha}} \mathrm{l o g} \, P ( y_{1}, \ldots, y_{L} \mid\mathbf{c} )=\frac{1} {L^{\alpha}} \sum_{t^{\prime}=1}^{L} \mathrm{l o g} \, P ( y_{t^{\prime}} \mid y_{1}, \ldots, y_{t^{\prime}-1}, \mathbf{c} ) ; $$ Note tha we have **6** candidates (A, C ..).
  <figure style="text-align: center;">
      <img alt="Max sequence" src="https://d2l.ai/_images/beam-search.svg" style="background-color: white; display: inline-block;">
      <figcaption> beam-search </figcaption>
    </figure>

## Chapter 11 : Attention Mechanisms and Transformers
- **Idea** : attention first come up in encoder - decoder design, rather than tranform the input to a fixed size feature and the feed it to all the decoder step, we want to create a representation that has the same length of input and decoder at each time step can pay attention to different input sequence (with it's weight). And transfromer give up residual connection, instead use attention at all.
- **Q, K, V**
  - Define database $\mathcal{D} \stackrel{\mathrm{d e f}} {=} \{( \mathbf{k}_{1}, \mathbf{v}_{1} ), \ldots( \mathbf{k}_{m}, \mathbf{v}_{m} ) \} $, give some query, the attention is $\text{Attention}(q, D) \stackrel{\mathrm{d e f}} {=} \sum_{i=1}^m \alpha(q, k_i)v_i $, where $\alpha(q, k_i)$ are scalar attention weight, this operation is also called attention pooling. We want this attention weight to be larger than 0 and sum up to 1, so we can use a softmax to transfrom it.
- **Attention pooling with similarity**
  - In a regression task, we can use kernel output as the attention weight (after normalization).
  - When directly compute loss $(f(x_i) - y_i)^2$， because we have $y_i$, the $\sigma$ will go to zero, causing overfitting. Even if we remove $y_i$ from the loss, if the dataset is large enough, we may still overfit.
  - <figure style="text-align: center;">
      <img alt="Attention Pooling" src="https://d2l.ai/_images/attention-output.svg" style="background-color: white; display: inline-block;">
      <figcaption> Attention Pooling </figcaption>
    </figure>
- **Attention Scoring Function**
  - **Dot Product Attention**: note that kernel of gaussian is $a(q, k_i) = q^{\intercal}k_i - 0.5 * ||q||^2 - 0.5 * ||k_i||^2$, and after the softmax normalization, second one is cancled out. Then if we get $k_i$ with batch or layer normalization, it's length will be bounded and often constant, so we can get rid of last term without penalty. This leads to $a(q, k_i) = q^{\intercal}k_i$, ann assume both $q$ and $k_i$ have zero mean and unit variance, this attention weight will have a variance of $d$ (which is query feature size), so we can further normalize it : $a(q, k_i) = q^{\intercal}k_i / \sqrt{d}$, this is the common one used in transformer. At last, we do a softmax on it.
  - Because we do not want consider \<pad\>, so we can do musk softmax. And for Batch Matrix Multiplication, use torch.bmm. bmm(Q, K) take Q(n, a, b) and K(n, b, c), which return [Q1 @ K1, ..., Q_n @ K_n].
  - **Additive Attention**: when q and k has different feature size, we can do a transform ($q^{\intercal}Mk$), or use this additive attention $a(\mathbf{q},\mathbf{k})=\mathbf{w}_v^\top\tanh(\mathbf{W}_q\mathbf{q}+\mathbf{W}_k\mathbf{k})\in\mathbb{R}$, inside the () we add by broadcasting.
- **Bahdanau Attention**
  - Attention function will work between encoder hidden state and decoder hidden state, $c_{t'} = \sum_t^T a(s_{t'-1}, h_t)h_t$, and this will used to generate $s_{t'}$.
  - <figure style="text-align: center;">
      <img alt="Bahdanau Attention" src="https://d2l.ai/_images/seq2seq-details-attention.svg" style="background-color: white; display: inline-block;">
      <figcaption> Bahdanau Attention </figcaption>
    </figure>
  - Seq2SeqAttentionDecoder first use encoder last layer hidden state as the query, and later use it's own hidden state from it's rnn model. Keys and values are all encoder outputs (last layer hidden state at all time step), then concatenate the embed X with this context (attention result) then feed to it's rnn.
- **Multi-head Attention**
  - <figure style="text-align: center;">
      <img alt="Multi-head Attention" src="https://d2l.ai/_images/multi-head-attention.svg" style="background-color: white; display: inline-block;">
      <figcaption> Multi-head Attention </figcaption>
    </figure>
  - We want the same Q, K, V to have different behaviour with the same attention mechanism, so we have to copy them $h$ times and first pass them into FC layer which has learnable param that can change the QKV, then feed to attention, get $h$ results, concatenate them. $h_i = f(W_i^{q}q_i, W_i^kk_i, W_i^vv_i)$, $f$ is attention pooling, each $W$ have shape of $(p_q, d_q), (p_k, d_k)$ and $(p_v, d_v)$, $h_i$ is of shape $(p_v,)$. We concatenate these h to a $(h \times p_v,)$ shape. And we use a big learnable matrix of shape $(p_o, h \times p_v)$ times the concatenated result, which finnally return output of shape $(p_o, )$. For the purpose of parallel computation, we set $hp_q = hp_k = hp_v = p_o$.
  - Impl of d2l is the same idea, but for parrallel, it use some trick, hidden_size is h * p_q, put num_head into batch_size, make the batch_size = batch_size * num_head, and in the output reverse it.
- **Self Attention**
  - Do the attention(X, X, X) to get encoder. Compare CNN, RNN and self-attention, given n sequence with d dimension. CNN : choose kernel of 3, computation is O(knd^2), longest connect path is O(n/k). RNN : compute O(nd^2), path O(n). Self attention : compute O(n^2d), path O(1).
  - Positional Encoding: self attention does not contain position (order) information, so a token at time step 1 and 5 are the same (but it should not!). So we need to add something to keep the position information. First we use fixed position encoding with sine and cosine. $X^{n \times d}$ is the input representation of n tokens, and the position encoding is $X + P$, where $p_{i,2j} = \sin \left( \frac{i}{10000^{2j/d}} \right)$ and $p_{i,2j+1} = \cos \left( \frac{i}{10000^{2j/d}} \right)$. This works because the function abouve contain different frequency information.
  - Relative position encoding : $\begin{bmatrix}
\cos(\delta\omega_j) & \sin(\delta\omega_j) \\
-\sin(\delta\omega_j) & \cos(\delta\omega_j)
\end{bmatrix}
\begin{bmatrix}
p_{i,2j} \\
p_{i,2j+1}
\end{bmatrix} = \begin{bmatrix}
p_{i+\delta,2j} \\
p_{i+\delta,2j+1}
\end{bmatrix}$, we can just add a (1, step, hidden) param, and add it to the embed(X) to learn the position.
- **Transformer**
  - <figure style="text-align: center;">
      <img alt="Transformer" src="https://d2l.ai/_images/transformer.svg" style="background-color: white; display: inline-block;">
      <figcaption> Transformer Arch </figcaption>
    </figure>
  - The encoder-decoder attention layer take decoder self-attention layer output as query, and encoder output as key and value.
  - Note that in decoder self-attention, we will carefully mask output to reserve the autoregreesive nature, we do not take position in the outpt (later as the input of decoder self-attention layer) after the position we are calculating.
  - Before pos encoding we first multiply sqrt(d) with embed(X) to rescale it, maybe because embed(X) has small variace.
  - For prediction, we need to cache the input X for the decoder, in training we can just compute all time step all together. In impl, it is cached in state[2].
  - **!!** only the last output of the encoder will do attention on all block of decoder.
- **Vision Transformer**
  - <figure style="text-align: center;">
      <img alt="Vision Transformer" src="https://d2l.ai/_images/vit.svg" style="background-color: white; display: inline-block;">
      <figcaption> Vision Transformer </figcaption>
    </figure>
  - patch embeding will feed to a conv then flatten it, return shape of (batch, patch, hidden)
  - Do the normalization before the attention is better for the efficient learning of transformer. The vit mlp layer use GELU and dropout is applied to the output of each fully connected layer in the MLP for regularization.

- Large Scale Pre-training
  - Encoder only, ViT, BERT. BERT use masked language modeling, and for a token, tokens at left and right can all attend to this masked token. So it is a bidirection encoder --- in the figure below, each token along the vertical axis attends to all input tokens along the horizontal axis.
  - <figure style="text-align: center;">
      <img alt="BERT" src="https://d2l.ai/_images/bert-encoder-only.svg" style="background-color: white; display: inline-block;">
      <figcaption> BERT </figcaption>
    </figure>
  - Encoder-Decoder, BART & T5, both attempt to reconstruct original text in their pretraining objectives, while the former emphasizes noising input (e.g., masking, deletion, permutation, and rotation) and the latter highlights multitask unification with comprehensive ablation studies.
  - <figure style="text-align: center;">
      <img alt="T5" src="https://d2l.ai/_images/t5-encoder-decoder.svg" style="background-color: white; display: inline-block;">
      <figcaption> T5 </figcaption>
    </figure>
  - Decoder only, GPT. **In-context learning** : conditional on an input sequence with the task description, task-specific input–output examples, and a prompt (task input). 
  - <figure style="text-align: center;">
      <img alt="GPT" src="https://d2l.ai/_images/gpt-decoder-only.svg" style="background-color: white; display: inline-block;">
      <figcaption> GPT </figcaption>
    </figure>
  - <figure style="text-align: center;">
      <img alt="x - shot" src="https://d2l.ai/_images/gpt-3-xshot.svg" style="background-color: white; display: inline-block;">
      <figcaption> x-shot </figcaption>
    </figure>
- Efficient Transformer design (see that survey)
  - Sparse attention：Longformer：使用滑动窗口和全局注意力，降低复杂度到O(n)。BigBird：结合随机注意力、窗口注意力和全局注意力，适合长序列。
  - Low rank approximation：Linformer：通过低秩分解将注意力矩阵投影到较低维度，复杂度从O(n²)降到O(n)。
  - Memory：Transformer-XL：引入循环记忆，处理长序列时重用之前的隐藏状态，避免重复计算。
  - Efficient attention：Performer：使用核方法（Favor+）近似点积注意力，复杂度降为O(n)。
  - Model compress：Distillation：将大Transformer蒸馏为小模型（如DistilBERT）。量化：减少参数精度，降低内存占用。
<!-- <img alt="ResNeXt Block" src="https://d2l.ai/_images/rnn.svg" style="background-color: white; display: inline-block;"> -->
<!-- <img alt="ResNeXt Block" src="https://d2l.ai/_images/rnn-bptt.svg" style="background-color: white; display: inline-block;"> -->

## Chapter 12 : Optimization
- **Convexity**
  - Convex set : for any $a, b \in \mathcal X$, given $\lambda \in [0, 1]$, $\lambda a + (1 - \lambda) b \in \mathcal X$.
  - Convex function : for any function $f : \mathcal X \rightarrow \mathbb R$, we have $\lambda f(x) + (1 - \lambda) f(x') >= f(\lambda x + (1-\lambda)x')$.
  - jensen's inequality : $\sum_i\alpha_if(x_i)\geq f\left(\sum_i\alpha_ix_i\right)$ and $E_X[f(X)]\geq f\left(E_X[X]\right)$
  - Properties : Local Minima Are Global Minima, below set $\mathcal{S}_b\overset{\mathrm{def}}{\operatorname*{=}}\{x|x\in\mathcal{X}\mathrm{~and~}f(x)\leq b\}$ is also a convex set, f is convex if hessian of f is positive semidefinite ($\nabla^2 f = H, x^THx >= 0$).
  - Convex with constraint $c_i(x) <= 0$, can be dealed with lagrangian, and the KKT condition. KKT are:
    - Stationarity : $\nabla_x L(f(x), \lambda_1, \ldots, \lambda_n) = 0$.
    - Primal Feasibility : $c_i(x) <= 0$
    - Dual Feasibility : $\lambda_i >= 0$
    - Complementary Slackness : $\lambda_i c_i(x) = 0$
  - Penality is robust than constraint. We can also use projection to satisfy constraints.
- **Gradient Descent**
  - $x \leftarrow x - \eta \nabla_x f(x)$, with newton's method $\eta = \nabla_x^{-2} f(x) = H^{-1}$
  - H is expensive, so we can use precondition $x \leftarrow x - \eta \text{diag}(H)^{-1}\nabla_x f(x)$, this means for different $x_i$ we use different learning rates.
  - Line search : use binary search to find $\eta$ that minimize $f(x - \eta \nabla_x f(x))$.
- **SGD**： converge with rate $\mathcal O (1/\sqrt T)$, $T$ is the sample number. More details of the math please see the book.
- **Momentum**
  - Use leaky average $v_k = \beta v_{k-1} + g_{k, k-1}$ as the gradient, this is the momentum!
  - Gradient descent with and without momentum for a convex quadratic function decomposes into coordinate-wise optimization in the direction of the eigenvectors of the quadratic matrix.
  - The velocity converge condition is loose than gradient converge condition, so add momentum (with big $\beta$ ) is theoritaly better.
- **Adagrad** : it is a SGD alg
  - Some features are rare, so we want to update it faster ( we do not update their gradient much ).
  - Some problem has large condition number $k = \lambda_{max} / \lambda_{min}$, which is not good. We can rescale them by some matrix (if Hessian of the problem L is possitive semidefinite), or just rescalse the diag of the Q. $\tilde Q = \text{diag}(Q)^{-1/2}Q\text{diag}(Q)^{-1/2}$. However this is not realistic in DL, because we don't have second derivitive of Q, so Adagrad use the norm of the gradient as the scalse item. And this makes it adjust element wise (like only diag will change).
  - $s_t = s_{t-1} + g_t^2, w_t = w_t - \eta / \sqrt{s_t+\epsilon} \odot g_t$, one problem of Adagrad is that it's learning rate decrease $\mathcal O(t^{-1/2})$.
- **RMSProp**
  - $s_t = \gamma s_{t-1} + (1-\gamma) g_t^2$, only difference with Adagrad
- **Adadelta**
  - $\mathbf{s}_{t}=\rho \mathbf{s}_{t-1}+(1-\rho) \mathbf{g}_{t}^{2}$, $\mathbf{g}_{t}^{\prime}=\frac{\sqrt{\Delta \mathbf{x}_{t-1}+\epsilon}}{\sqrt{\mathbf{s}_{t}+\epsilon}} \odot \mathbf{g}_{t}$, $x_t = x_t - \mathbf{g}_{t}^{\prime}$, $\Delta\mathbf{x}_{t}=\rho\Delta\mathbf{x}_{t-1}+( 1-\rho) \mathbf{g}_{t}^{\prime\, 2}, $.
- **Adam**
  - $v_t = \beta_1 v_{t-1} + (1-\beta_1)g_{t}$, $s_t = \beta_2 s_{t-1} + (1-\beta_2)g^2_{t}$, and the rescale it (otherwise the initial numbers are too diverge from gradient), $\hat v_t = v_t / (1 + \beta_1^t)$, $\hat s_t = s_t / (1 + \beta_2^t)$. Then finnally $x_t = x_t - \eta \hat v_t / (\sqrt{\hat s_t} + \epsilon)$
  - One of the problems of Adam is that it can fail to converge even in convex settings when the second moment estimate in $s_t$ blows up as $g^2_t$ being too large and forget the history. Yogi update is $s_t = s_{t-1} + (1-\beta_2)g^2_{t} \odot (g^2_{t} - s_{t-1})$, the update is not the deviation of $g^2_{t} - s_{t-1}$, it is $g^2_{t}$ with regard to the sign.
- Scheduler
  - Warmup: In particular they find that a warmup phase limits the amount of divergence of parameters in very deep networks. A closer look at deep learning heuristics: learning rate restarts, warmup and distillation. ArXiv:1810.13243.
<!-- <img alt="ResNeXt Block" src="https://d2l.ai/_images/rnn.svg" style="background-color: white; display: inline-block;"> -->
<!-- <img alt="ResNeXt Block" src="https://d2l.ai/_images/rnn-bptt.svg" style="background-color: white; display: inline-block;"> -->

## Chapter 13 : Computation
- compiler
  - net = torch.jit.script(net)
- Automatic Parallesim
  - y.to('cpu', non_blocking=non_blocking) for y in x, will return x[i-1] when calculate x[i]
- Tranning on multiple GPU
  - <figure style="text-align: center;">
      <img alt="Partion Methods" src="https://d2l.ai/_images/splitting.svg" style="background-color: white; display: inline-block;">
      <figcaption> Partion Methods </figcaption>
    </figure>
  - nn.parallel.scatter to split data to different devices
  - 显式同步（torch.cuda.synchronize()）仅在需要精确测量执行时间或调试异步错误时必要，其他情况会自己根据cpu或者后续数据需求隐式调用
- Concise impl :
  - What we need to do
    - Network parameters need to be initialized across all devices.
    - While iterating over the dataset minibatches are to be divided across all devices.
    - We compute the loss and its gradient in parallel across devices.
    - Gradients are aggregated and parameters are updated accordingly.
  - Use torch.nn.parallel.DistributedDataParallel
- Parameter Server
  - <figure style="text-align: center;">
      <img alt="Parameter Exchange" src="https://d2l.ai/_images/ps-distributed.svg" style="background-color: white; display: inline-block;">
      <figcaption> Parameter Exchange </figcaption>
    </figure>
  - last graph above assume gradient can be divided into four parts, and exchange each one of them each GPU.
  - Ring Synchronization
  - Key–Value Stores

<!-- <img alt="ResNeXt Block" src="https://d2l.ai/_images/rnn.svg" style="background-color: white; display: inline-block;"> -->
<!-- <img alt="ResNeXt Block" src="https://d2l.ai/_images/rnn-bptt.svg" style="background-color: white; display: inline-block;"> -->

## Chapter 14 : Computer Vision
- How to compute the reception field : from top to bottom. $r_i = (r_{i+1}-1) * s_i + k_i$.
- Image Augmantation
  - torchvision.transforms.RandomHorizontalFlip, RandomResizedCrop, ColorJitter... and use Compose to combine them. Use ToTensor to convert to tensor.
- Fine tunning
  - <figure style="text-align: center;">
      <img alt="Fine tunning" src="https://d2l.ai/_images/finetune.svg" style="background-color: white; display: inline-block;">
      <figcaption> Fine tunning </figcaption>
    </figure>
  - Change the output layer, random init it, then use a small lr to train it (Output layer can have bigger lr than other layers).
  - torchvision.datasets.ImageFolder get image dataset
- Anchor Box :
  - from a pixel get (s1, r_i(up to m)) and (s_i(up to n), r1), s is scale, r is width / height, so we have wh(n+m-1) anchor boxes, w and h is the img size. Anchor width and height is $hs\sqrt r$ and $hs / \sqrt r$. Here we assume that h == w, if it is not the case then $hs\sqrt r$ and $ws/\sqrt r$.
  - IoU, Inter over Union.
  - How to assign ground truth to anchor box? Given anchors $A_1, \ldots, A_{n_c}$, and ground truth $B_1, \ldots, B_{n_t}$. We have a matrix $X \in \mathbb R^{n_c \times n_t}$, $X_{i, j}$ is IoU of $A_i$ and $B_j$. Find max number in $\max X = X_{i_1, j_1}$, assign $B_{j_1}$ to $A_{i_1}$, abandon $i_1$ row and $j_1$ col. Find again and again until we finish assign B. For left A, find in it's row max B, if the IoU is bigger than some threshold, assign B to A.
  - We then compute the offset with $\left( \frac{\frac{x_{b}-x_{a}} {w_{a}}-\mu_{x}} {\sigma_{x}}, \frac{\frac{y_{b}-y_{a}} {h_{a}}-\mu_{y}} {\sigma_{y}}, \frac{\operatorname{l o g} \frac{w_{b}} {w_{a}}-\mu_{w}} {\sigma_{w}}, \frac{\operatorname{l o g} \frac{h_{b}} {h_{a}}-\mu_{h}} {\sigma_{h}} \right) $, mean set to 0, and $\sigma_{y} = \sigma_{x} = 0.1, \sigma_{w} = \sigma_{h} = 0.2$. Normalization will make the training more stable. And we do not compute offset for IoU < threshold anchors, so we use a mask to filter it. For other anchors, we have predict_offset that will compare with offsets for tranining.
  - non-maximum suppression，NMS: we sort all the anchors with it's class predict accuracy. In a loop, choose max one, delete all the other anchors with IoU (with the choosen anchor) larger than a threshold. We can also first erase all the anchors based on some threshold to decrease computational cost.
- Single Shot Multibox Detection
  - <figure style="text-align: center;">
      <img alt="SSD" src="https://d2l.ai/_images/ssd.svg" style="background-color: white; display: inline-block;">
      <figcaption> SSD </figcaption>
    </figure>
  - Use Conv2d rather than FC to get prediction. Flatten different feature map on the dim != 0 (batch_size), then cat them so we have connect different feature map. nn.Conv2d(num_inputs, num_anchors * (num_classes + 1), kernel_size=3, padding=1) for class data and nn.Conv2d(num_inputs, num_anchors * 4, kernel_size=3, padding=1) for offset.
- Region-based CNNs (R-CNNs)
  - RCNN:
    1. Use a select algorithm to select region proposals from the input image and resize it (to match the cnn below)
    2. pass all the regions to a pre trained CNN to get the feature (it is slow!)
    3. Use these feature train n_class + 1 SVM
    4. combine the feature and the box as a sample, train a linear regression model
    5. <figure style="text-align: center;">
          <img alt="RCNN" src="https://d2l.ai/_images/r-cnn.svg" style="background-color: white; display: inline-block;">
          <figcaption> RCNN </figcaption>
        </figure>
  - Fast RCNN
    1. still use select algorithm, but just go through CNN once, and project the proposal to the feature map.
    2. use ROI Pooling to convert these feature maps to the same output size (7 * 7), roi pool is max pool. ROI pool will first divide the region into 7 * 7 regions and then quantize, because the region of interest in the feature map has corrdinate in float type, but what we need is int type. So quantize them to int, and do the max pool, we get 7 * 7 output. There are 2 quantization, first the feature map corrdinates and the division.
    3. then use FC to get two branch : one for N(n_class + 1), and 4N for the box.
    4. <figure style="text-align: center;">
          <img alt="RCNN" src="https://d2l.ai/_images/fast-rcnn.svg" style="background-color: white; display: inline-block;">
          <figcaption> Fast RCNN </figcaption>
        </figure>
  - Faster RCNN
    1. use a RPN net (region proposal network) : use a pad = 1, k = 3 conv to extract new feature from the origin feature from the base net. Then get anchors from this new feature map, get class predict (! this is not the full class predict, it is only binary class predict -- background or not) and box predict, go through NMS and fuse with origin feature.
    2. <figure style="text-align: center;">
          <img alt="Faster RCNN" src="https://d2l.ai/_images/faster-rcnn.svg" style="background-color: white; display: inline-block;">
          <figcaption> Faster RCNN </figcaption>
        </figure>
  - Masked RCNN
    1. If we have detailed pixel information of the target, we can replace ROI pool with region of interest alignment, use bipolar interpolation to preserve spatial information. This can then use a FC to get class and box, or use a conv layer to get pixel detailed information of the origiin img (segment).
    2. ROI Align: compare with ROI pool, we do not quantize, instead we use samples. We divede the region in to the output shape we want (in float type), and for each cell do another division. For example, if we set hyperparameter sample size to $n \times n$, then divede the cell to $n \times n$. For each cell in the cell, calculate the value based on bipolar interpolation: $$F(x,y) = \sum_{a,b} F(\lfloor x \rfloor + a, \lfloor y \rfloor + b) \cdot \max(0, 1-|x-(\lfloor x \rfloor + a)|) \cdot \max(0, 1-|y-(\lfloor y \rfloor + b)|)$$ for $a, b \in \{0, 1\}$. Finally, get the average on $n \times n$ cell in cell, get the cell value.
    3. If you interested in more, we have a precise ROI pool: same as ROI align we do not quantize, but we do not do the divide into samples too. Instead we do a integral on the cell.
    4. <figure style="text-align: center;">
          <img alt="Masked RCNN" src="https://d2l.ai/_images/mask-rcnn.svg" style="background-color: white; display: inline-block;">
          <figcaption> Masked RCNN </figcaption>
        </figure>
- semantic segmentation
  - image segmentation do not care about the semantic, semantic segmentation cares (it will distinguish dog and cat), instance segmentation also called simultaneous detection and segmentation not only has semantic information but also distinguish different object (two dogs and three cats are different).
  - semantic data augmentation will not scale, only crop
- Transposed Convolution
  - Conv animation : [https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md](conv_animation)
  - Normal convolution will downsample the height and width, transposed one will increase it, it is the gradient of the std convolution. (but tconv(conv(x)) != x, there is imformation loss)
  - Y[i: i + h, j: j + w] += X[i, j] * K and the torch function is nn.ConvTranspose2d
  - Just see the code and it's comment below.
  - <figure style="text-align: center;">
      <img alt="Transposed Convolution" src="https://d2l.ai/_images/trans_conv.svg" style="background-color: white; display: inline-block;">
      <figcaption> Transposed Convolution </figcaption>
    </figure>
- Fully Convolutional Networks
  - In order to give every pixel it's class predict, we can use transposed convolution to transform the feature back into the original shape.
  - We use a pretrained ResNet base net, get rid of last two layers (an adaptive avg pool and a linear), connect it to 1 * 1 conv kernel with channel == num_classes, and a transposed convolution layer.
  - We often use bi-linear interpolation to initialize the transposed convolution layer. See in test.ipynb.
- Neural Style Transfer
  - We need two inputs, one content image, one style image. We use a pretrained CNN base model as the feature extractor, some layers will be use as content image feature, some as style image feature. The model param is frozen!!! The only parameter will change is the synthesized image. We have three loss, the total variation loss is to reduce the noise.
  - In the example in book, we use deeper layer as the content layer (do not focus on detail), and different depth layer for style layer (focus both detail and global).
  - Loss: for content loss we use torch.square(Y_hat - Y.detach()).mean(), stop the gradient flow throgh the target content image Y, it is a stated value, not a variable (and Y_hat is content_layer(content_x)). For style loss, we use the gram matrix to get correlation of the style features of different channels, for a [n, c, h, w] feature, reshape it to [c, nhw] matrix X and calculate G = XX^T, loss = torch.square(gram(Y_hat) - gram_Y.detach()).mean(). For total variation loss, $\sum_{i, j}|x_{i,j} - x_{i+1,j}|+|x_{i,j} - x_{i,j+1}|$, this will reduce the noise. Add all the content loss and style loss in all the correspoding layers with the weight and the total loss with it's weight we get the total loss. Note in book example, X, styles_Y_gram, trainer = get_inits(X, device, lr, styles_Y) the first X (target data) is a copy of second X (origin content img), it is two different data.
  - <figure style="text-align: center;">
      <img alt="neural-style" src="https://d2l.ai/_images/neural-style.svg" style="background-color: white; display: inline-block;">
      <figcaption> Neural Style </figcaption>
    </figure>
<!-- <img alt="ResNeXt Block" src="https://d2l.ai/_images/rnn.svg" style="background-color: white; display: inline-block;"> -->
<!-- <img alt="ResNeXt Block" src="https://d2l.ai/_images/rnn-bptt.svg" style="background-color: white; display: inline-block;"> -->

## Chapter 15 : NLP Pretraining
- Word Embedding (word2vec)
  - one-hot is not good, we can not get similarity from it (for example cosine similarity).
  - Skip-Gram Model:
    - use one word (center word) to generate other word (context word) with a probability, each word has two vectors to represent itself, $v_i$ when use as center word, $u_i$ when use as context word.
    - Here we use: $P(w_o | w_c) = \frac{\exp(u_o^{\top}v_c)}{\sum_{i \in \mathcal V} \exp(u_i^{\top}v_c)}$. And given a sequence of length $T$, a context window $m$, the skip-gram model is $\prod_{t=1}^T\prod_{-m \leq j \leq m, j \neq 0} \log P(w^{t+j}|w^t)$. Note that index less than 0 or larger than T will be omitted.
    - Maximize this model, we get the trained params (v and u), and in nlp, v (center word representation) is often used as word representation.
  - The Continuous Bag of Words (CBOW) Model
    - different from Skip-Gram, CBOW center word is generated from context words.
    - Switch the meaning of $v$ and $u$, $P(w_c\mid w_{o_1},\ldots,w_{o_{2m}})=\frac{\exp\left(\frac{1}{2m}\mathbf{u}_c^\top(\mathbf{v}_{o_1}+\ldots+\mathbf{v}_{o_{2m}})\right)}{\sum_{i\in\mathcal{V}}\exp\left(\frac{1}{2m}\mathbf{u}_i^\top(\mathbf{v}_{o_1}+\ldots+\mathbf{v}_{o_{2m}})\right)}$, denote $\mathcal W_o = \{w_{o1}, \dots, w_{o2m}\}$ and $\bar v_o = (v_{o1} + \dots + v_{o2m}) / 2m$ then we get $P(w_c \mid \mathcal W_o) = \frac{\exp(u_c^{\top} \bar v_o)}{\sum_{i \in \mathcal V} \exp(u_i^{\top} \bar v_o)}$
    - Given a sequence with length $T$, the model is $\prod_{t=1}^T P(w^t \mid w^{t-m}, \dots, w^{t+m})$, after traning, it will use the context word vector as the word representation.
- Approximate Training
  - In the softmax above, the denominator containt all the vectors from the vocabulary, so the gradient calculation is hard to get, we need to approximate it.
  - Negative Sampling:
    - given a center word $w_c$, any context word $w_o$ coms from the context window is considered as an event: $P(D=1 \mid w_c, w_o) = \sigma(u_o^{\top}v_c)$, $\sigma$ use sigmoid activation function. So we train to maximize $\prod_{t=1}^T\prod_{-m \leq j \leq m, j \neq 0} \log P(D=1|w^t, w^{t+j})$.
    - But this only contains positive items, we need to add negitive ones. Change the above $P(D=1|w^t, w^{t+j})$ to $P(D=1|w^t, w^{t+j}) \prod_{k=1, w_k \sim P(w)}^K P(D=0 \mid w^t, w_k) \rightarrow P(w^{t+j} \mid w^t)$, where $P(w)$ is a predefined distribution that will sample $K$ noise words that is negitive (not in the context window).
    - so $-\log P(w^{t+j} \mid w^t) = -\log \sigma\left( \mathbf{u}_{i_t+j}^{\top} \mathbf{v}_{i_t} \right) - \sum_{k=1, \, w_k \sim P(w)}^{K} \log \sigma\left( -\mathbf{u}_{h_k}^\top \mathbf{v}_{i_t} \right)$, only depend on the hyperparameter $K$, not vocab size involve.
  - Hierarchical Softmax:
    - Use a binary tree where leaf node represents a word. $L(w)$ is the path length from root to leaf represents word $w$, $n(w,j)$ is the $j$th node in this path with it's context word vector $u_{n(w,j)}$.
    - We use $P(w_o \mid w_c) = \prod_{j=1}^{L(w_o)-1} \sigma \left( \left[ n(w_o, j+1) = \text{leftChild}(n(w_o, j)) \right] \cdot \mathbf{u}_{n(w_o, j)}^\top \mathbf{v}_c \right)$, $[*]$ return 1 if true, otherwise -1.
    - since $L(w_o) - 1$ is on the order of $\mathcal O(\log_2|\mathcal V|)$, when vocab size is huge, this can reduce the traning cost.
- Dataset
  - Subsampling: some word like 'a', 'to' is too common, it's context word varies, so we want to sub-sample (discard) them with a probability $P(w_i) = \max(1 - \sqrt{t / f(w_i)}, 0)$, where $f(w_i)$ is the ratio of word $w_i$.
  - Minibatch: $i^{th}$ example include a center word and its $n_i$ context words and $m_i$ noise words. For different $i$, $n_i + m_i$ varies, so we pad them with 0, so for the loss calculation, we need a mask. And for distinguish between context word and noise word, we need another variable as label (work like mask).
  - When the corpus is huge, we often sample context words and noise words for the center words in the current minibatch.
- Word Embedding with Global Vectors (Glove)
  - Start from skip-gram: let $x_{ij}$ be times $w_i$ (as the center word) and $w_j$ (as the context word) occur in corpus in certain window size, the loss is the same as $-\sum_i\sum_jx_{ij}\log q_{ij}$, here $q_{ij} = p(w_j \mid w_i)$. Now write $p_{ij} = x_{ij} / x_i$, we get $-\sum_ix_i\sum_jp_{ij}\log q_{ij}$, which takes the form of a weighted cross entropy loss.
  - However, the softmax has the problem we mentioned above, and cross entropy may assign large weight to rare events. So GloVe makes three changes:
    - Do not use probability distribution: $p_{ij}^{'} = x_{ij}, q_{ij}^{'} = \exp(u_j^{\top}v_i)$, and use **Squared Loss** $\left(\log p_{ij}^{'} - \log q_{ij}^{'} \right)^2 = \left(u_j^{\top}v_i - \log x_{ij} \right)^2$
    - Add two bias to the center word and context word parameters, $b_i$ and $c_i$.
    - replace the weight to $h(x_{ij})$ which increase in $[0, 1]$.
    - Then we get the loss $=-\sum_i\sum_jh(x_{ij})\left(u_j^{\top}v_i + b_i + c_j - \log x_{ij} \right)^2$. $h(x) = (x/c)^{\alpha}$ ($\alpha = 0.75$, $c = 100$) if $x < c$ else 1 is a suggestion.
  - The model is symmetric as $x_{ij} = x_{ji}$, so the two vector of word is mathmetically same, but during traning they may get different value (initialization), so the **word vector** is sum of them.
  - Other way to get the same loss function: consider ratio of co-occurrence probability $p_{ij} / p_{ik}$, we want some function $f(u_j,u_k,v_i)$ to fit it, based on the exchange of $j$ and $k$, we find $f(x)f(-x)=1$, so one posibility is $f = \frac{\exp(u_j^{\top}v_i)}{\exp(u_k^{\top}v_i)}$, let $\exp(u_j^{\top}v_i) \approx \alpha p_{ij}$, we get $u_j^{\top}v_i + b_i + c_j - \log x_{ij} \approx 0$
  - **GloVe if build from counting, and skip-gram and CBOW is build from probability**.
- Subword Embedding
  - fastText model: add \< and \> to a word, then divide the word into different length subword. For a word $w$, let $\mathcal G_w$ the union of all its subword of length between 3 and 6 and its special subword (\<word w\>). Then the word vector of it is $v_w = \sum_{g \in \mathcal G_w}z_g$.
  - Byte Pair Encoding (BPE): iteratively merge most frequent pairs, so we can control the size of the vocab.
  - fastText and GloVe, 2 web of pretrained word embeddings.
- Bidirectional Encoder Representations from Transformers (BERT)
  - The above word embedding method is context-independent, namely f(x) is only a function of word x, this is a limitation as the same word in different context may have different meanings. So we have context-sensitive representations like TagLM, CoVe, and ELMo, f(x, c(x)). ELMo combing all the middle layer of the bidirection LSTM as its output, and serve as additional feature for downstream tasks, during the downstream task training, the ELMo layers are frozen.
  - But these models is task-specific. We need to design a architecture for every NLP tasks, this is hard. GPT based on Transformer decoder build a task-agnostic model, its output will be passed to a output layer, and fine-tune the model for the downstream task. However it can only look forward because of the natural of autoregressive, so for the same word the context may differ in the future, so does the meaning.
  - BERT combine them, can see the left and right context, and same as GPT fine-tune, fed to a output layer (train from scrath).
  - <figure style="text-align: center;">
      <img src="https://d2l.ai/_images/elmo-gpt-bert.svg" alt="nlp embedding" style="background-color: white; display: inline-block;"/>
      <figcaption> A comparison of ELMo, GPT, and BERT </figcaption>
    </figure>
  - BERT use [cls] at the top, and then a sequence, then a [seq] spetial token. In order to fit different tasks, there are two way to construct the BERT input. 1 is [cls] sequnces [seq] for task like emotion analyse, 2 is [cls] sequences 1 [seq] sequences 2 [seq] for task like nautral language inference. And to distinguish text pairs, BERT add a learned segment embedding $e_A$ and $e_B$ to the token embedding, see figure below. And BERT use a learned positional embedding.
  <!-- - <img alt="BERT input" src="https://d2l.ai/_images/bert-input.svg" style="background-color: white; display: inline-block;"> BERT input -->
  - <figure style="text-align: center;">
      <img src="https://d2l.ai/_images/bert-input.svg" alt="BERT input" style="background-color: white; display: inline-block;"/>
      <figcaption> BERT input </figcaption>
    </figure>
  - Pretraining Tasks:
    - Masked Language Modeling: get the BERT output, add mask on certain position, add a mlp layer to predict these position. The mask can be a special token [\<mask\>] or the original token or a randome token, we can use a probability of [0.8, 0.1, 0.1] for all three cases. Note than special token [cls] and [sep] will not be the prediction target.
    - Next Sentence Prediction: MLM does not understant the relationship of two sequences. So we implement this, add one layer after [cls] token representation to train a binary task (whether next sequence is logically the next one for this sequence). In training, we sample 50% ramdom sequence and 50% true next sequence.
    - BERT pretraning is a linear combination of these 2 tasks.

## Chapter 16 : NLP Applications
<figure style="text-align: center;">
  <img alt="nlp-map-app" src="https://d2l.ai/_images/nlp-map-app.svg" style="background-color: white; display: inline-block;">
  <figcaption> NLP Applications </figcaption>
</figure>

- Sentiment Analysis using RNN
  - Glove (embedding, frozen) -> Bidirection RNN (encoder) -> linear (decoder)
- Using CNN:
  - *One dimensional correlation of mutiple channel is same as two dimensional corelation*.
  - *Max-over-time Pool*: pool the sequence into certain width (different channel no operation).
  - <figure style="text-align: center;">
      <img alt="textcnn" src="https://d2l.ai/_images/textcnn.svg" style="background-color: white; display: inline-block;">
      <figcaption> Text CNN </figcaption>
    </figure>
- Natural Language Processing:
  - Three type: entailment (positive next), contradiction (negitive next), neutral (not next).
  - Given premise and hypohesis tokens $A = (a_1,  \ldots, a_m), B = (b_1,  \ldots, b_m)$, where $a_i, b_j \in \mathbb R^d$ is word vector. The attenton is $e_{ij} = f(a_i)^{\top}f(b_j)$, $f$ is a mlp layer. We define the soft aligned representation:
    $$\boldsymbol{\beta}_i = \sum_{j=1}^{n}\frac{\exp(e_{ij})}{ \sum_{k=1}^{n} \exp(e_{ik})} \mathbf{b}_j.$$
    $$\boldsymbol{\alpha}_j = \sum_{i=1}^{m}\frac{\exp(e_{ij})}{ \sum_{k=1}^{m} \exp(e_{kj})} \mathbf{a}_i.$$
    Then we compare this representations with word vectors (again $g$ is a mlp layer):
    $$\begin{split}\mathbf{v}_{A,i} = g([\mathbf{a}_i, \boldsymbol{\beta}_i]), i = 1, \ldots, m\\ \mathbf{v}_{B,j} = g([\mathbf{b}_j, \boldsymbol{\alpha}_j]), j = 1, \ldots, n.\end{split}$$
    Then we aggregating the information:
    $$\mathbf{v}_A = \sum_{i=1}^{m} \mathbf{v}_{A,i}, \quad \mathbf{v}_B = \sum_{j=1}^{n}\mathbf{v}_{B,j}.$$
    and then feed them into a mlp layer then get the classification result: $\hat{\mathbf{y}} = h([\mathbf{v}_A, \mathbf{v}_B]).$
- Fine-tune BERT: single text, text pair, text tag (on every token representation add a shared dense layer), Q&A (first sequence as question, second as passage, from passage find out answer for question, we also need additional two output layer for predicting start pos and end pos, and the prediction is[start, end] pair).
<!-- <img alt="ResNeXt Block" src="https://d2l.ai/_images/rnn.svg" style="background-color: white; display: inline-block;"> -->
<!-- <img alt="ResNeXt Block" src="https://d2l.ai/_images/rnn-bptt.svg" style="background-color: white; display: inline-block;"> -->

## Chapter 17 : Reinforcement Learning
- Markov Decision Process
  - $\mathcal S, \mathcal A$ are states and actions a robot can take, when taking an action, states after may not
be deterministic, it has a probability. We use a transition function $T: \mathcal S \times \mathcal A \times \mathcal S \rightarrow [0, 1]$ to denote this, $T(s, a, s') = P(s' | s,a)$ is the probability of reaching s' given $s$ and $a$. For $\forall s \in \mathcal S$ and $\forall a \in \mathcal A$, $\sum_{s^{'}\in S}T(s, a, s^{'}) = 1$. Reward $r:\mathcal S \times \mathcal A \rightarrow \mathbb R$, $r(s,a)$.
  - This can build a MDP, $(\mathcal S, \mathcal A, \mathcal T, r)$, and a trajectory $\tau = (s_0, a_0, r_0, s_1, a_1, r_1, s_2, a_2, r_2, \ldots)$. The *return* of a trajectory is the total reword $R(\tau) = \sum_t r_t$, the goal of reinforcement learning is to find a trajectory that has the largest return. The trajectory might be infinite, so in order for a meaningful formular of its return, we introduce a discount factor $\gamma < 1$, $R(\tau) = \sum_{t=0}^{\infty}\gamma^tr_t$. For large $\gamma$, the robot is encouraged to explore, for small one to take a short trajetory to goal.
  - Markov system only depend on current state and action, not the history one (but we can always augment the system).
    
- Value Iteration
  - *Value function* is the value of a state, from that state, the expected sum reward (return). *Action Value function* is the value of an action at state s, from that state, take that action, the expected sum reward (return). And $V^{\pi}(s) = \sum_{a \in \mathcal A}\pi(a\mid s) Q^{\pi}(s,a)$.
  - Policy $\pi(s) = \pi(a\mid s)$, and $\sum_a \pi(a \mid s) = 1$. Given a policy, we may get a lot of trajectories, the average return of those are $$V^{\pi}(s_0) = E_{a_t \sim \pi(s_t)}[R(\tau)] = E_{a_t \sim \pi(s_t)}\left[ \sum_{t=0}^{\infty}\gamma^tr(s_t, a_t) \right]$$ this is the *value function* of policy $\pi$. Given two probability (one for choose the action, one for take the action), we can write the probability of a certain trajectory as $$P(\tau) = \pi(a_0\mid s_0) \cdot P(s_1 \mid s_0, a_0) \cdot \pi(a_1\mid s_1) \cdot P(s_2 \mid s_1, a_1) \cdots$$ So if we divide the trajectory into 2 parts, $s_0$ and $\tau^{'}$, we get $$R(\tau) = r(s_0, a_0) + \gamma \sum_{t=1}^{\infty}\gamma^{t-1}r(s_t, a_t) = r(s_0, a_0) + \gamma R(\tau^{'})$$
And the expectaion of this is (the optimal policy is the argmax of this):
$$
\begin{array}{lll}
V^{\pi}(s_0) &=& E_{a_t \sim \pi(s_t)}[r(s_0, a_0) + \gamma R(\tau^{'})] \\
             &=& E_{a_0 \sim \pi(s_0)}[r(s_0, a_0)] + E_{a_0 \sim \pi(s_0)}\left[E_{s_1 \sim P(s_1 \mid a_0, s_0)}[E_{a_t \sim \pi(s_t)}[\gamma R(\tau^{'})]]\right] \\ 
             &=& E_{a_0 \sim \pi(s_0)}[r(s_0, a_0)] + E_{a_0 \sim \pi(s_0)}\left[E_{s_1 \sim P(s_1 \mid a_0, s_0)}[V^{\pi}(s_1)]\right] \\
             &\rightarrow & \\
V^{\pi}(s)   &=& \sum_{a \in \mathcal A}\pi(a\mid s)\left[r(s,a) + \gamma \sum_{s^{'}}P(s^{'} \mid s, a)V^{\pi}(s^{'})\right], \forall s \in S \\
             &\rightarrow & r(s,a) = \sum_r p(r \mid s,a)r = \sum_r\sum_{s^{'}}p(r,s^{'} \mid s,a)r =  \sum_{s^{'}}p(s^{'} \mid s,a)r(s^{'})\\
             &=& \sum_{a \in \mathcal A}\pi(a\mid s)\sum_{s^{'} \in \mathcal S}P(s^{'} \mid s, a)\left[r(s^{'}) + \gamma V^{\pi}(s^{'})\right]
\end{array}$$
Last arrow please refer to *Math of Reinforcement Learning*, if we assume reward only depend on next state.
  - *Action Value* (start at $s_0$ but action is fixed to $a_0$):
$$
\begin{array}{lll}
Q^{\pi}(s_0, a_0) &=& r(s_0, a_0) + E_{a_t \sim \pi(s_t)} \left[ \sum_{t=1}^{\infty}\gamma^{t}r(s_t, a_t) \right] \\
                  &\rightarrow & \\
Q^{\pi}(s, a)     &=& r(s,a) + \gamma \sum_{s^{'}}P(s^{'} \mid s, a)\sum_{a^{'}}\pi(s^{'} \mid a^{'})Q^{\pi}(s^{'}, a^{'}), \forall s \in S, a \in \mathcal A \\
                  &=& r(s,a) + \gamma \sum_{s^{'}}P(s^{'} \mid s, a)V^{\pi}(s^{'}), \forall s \in S, a \in \mathcal A
\end{array}
$$
We can also write it into the form of $r(s^{'})$, which is the form Gym takes.
  - Value Iteration: initialize the value function to arbitrary values $V_0(s), \forall s \in S$. If the policy is deterministic, then at $k^{th}$ iteration (max will return one action, which means the action policy is deterministic): $$V_{k+1}(s) = \max_{a \in \mathcal A} \left \{ r(s,a) + \gamma \sum_{s^{'}}P(s^{'} \mid s, a)V_{k}(s^{'}) \right \}, \forall s \in \mathcal S$$ And $V^*(s) = \lim_{k \rightarrow \infty} V_k(s)$. The same iteration can be written with action value: $$Q_{k+1}(s,a) = r(s,a) + \gamma \max_{a^{'} \in A}\sum_{s^{'}}P(s^{'} \mid s, a)Q_k(s^{'}, a^{'})$$
  - Policy Evaluation: if the policy is not deterministic, it is stochastic, we can also get:
$$V_{k+1}^{\pi}(s) = \sum_{a \in \mathcal A}\pi(a \mid s) \left [ r(s,a) + \gamma \sum_{s^{'} \in \mathcal S}P(s^{'} \mid s, a)V_{k}^{\pi}(s^{'}) \right ], \forall s \in \mathcal S$$

- Q-Learning
  - Using the robot's policy $\pi_e(a \mid s)$, we can approximate the value iteration:
    $$\hat Q = \min_Q \frac{1}{nT}\sum_{i=1}^n\sum_{t=0}^T\left(Q(s_t^i, a_t^i)-r(s_t^i, a_t^i)-\gamma \max_{a^{'}}Q(s_{t+1}^i, a^{'})\right)^2 = \min_Q \mathscr l(Q)$$
    Using gradient descent:
    $$
    \begin{array}{lll}
    Q(s_t^i, a_t^i) &\leftarrow& Q(s_t^i, a_t^i) + \alpha \nabla_{Q(s_t^i, a_t^i)} \mathscr l (Q) \\
                    &=         & (1-\alpha)Q(s_t^i, a_t^i) +\alpha\left(r(s_t^i, a_t^i) + \gamma \max_{a^{'}}Q(s_{t+1}^i, a^{'})\right)
    \end{array}
    $$
    In order to stop at terminal state, we update the formular:
    $$Q(s_t^i, a_t^i) = (1-\alpha)Q(s_t^i, a_t^i) +\alpha\left(r(s_t^i, a_t^i) + \gamma (1 - \mathbb 1_{s_{t+1}^i \text{is terminal}}) \max_{a^{'}}Q(s_{t+1}^i, a^{'})\right)$$
    Then we get $\hat \pi(s) = \arg \max_a \hat Q(s,a)$.
  - In order to explore well, with current estimate of Q, set the policy to (*$\epsilon$-greedy exploration policy*):
    $$ \pi_e(a \mid s) = \left \{ 
    \begin{array}{lll}
    \arg \max_{a^{'}} \hat Q(s, a^{'}) & \text{with prob.} ~1 - \epsilon\\
    \text{uniform}(\mathcal A)         & \text{with prob.} ~\epsilon
    \end{array}
    \right .$$
    Other choise is *softmax exploration policy*, $T$ is called *temperature*:
    $$\pi_e(a \mid s) = \frac{e^{\hat Q(s,a)/T}}{\sum_{a^{'}}e^{\hat Q(s,a^{'})/T}}$$

## Chapter 18 : Gaussian Processes
- Definition: any subset of a GP is multivariate Gaussian distribution. It is defined solely on its mean and covariance function.
  - For a given mean function $m(x)$, a covariance function $k(x,x^{'})$, noise $w \sim \mathcal N(0, \sigma^2)$, conditioned on Data $(X,y)$, for new data point $x^*$, we have:
    $$ 
    \begin{array}{lll}
    \mu(x^*) &=& m(x^*) + k(x^*, X) \left [ k(X,X) + \sigma^2 I\right]^{-1}(y - m(x^*)) \\
    v(x^*)   &=& k(x^*,x^*) - k(x^*, X)\left [ k(X,X) + \sigma^2 I\right]^{-1}k(X, x^*)
    \end{array}
    $$
- Priors:
  - For any function with linear parameters, which takes this form $f(x) = \sum_i w_i \phi_i(x)$, is a GP, where $w \sim \mathcal N(u, S)$. And the mean function is $m(x) = u^{\top}\phi(x)$, covariance function is $k(x, x^{'}) = \phi(x)^{\top}S\phi(x)$. Any GP can take this form, but $\phi(x)$ maybe infinite dimension, for example, RBF kernel.
    $$
    \begin{array}{lll}
    k(x, x^{'}) &=& E\left[(f(x)-E[f(x)])(f(x^{'})-E[f(x^{'})])\right] \\
                &&\rightarrow  E[f(x)] = E[W^T\phi(x)] = U^T\phi(x), W=[w_0, w_1, \ldots], U = [u_0, u_1, \ldots] \\
                &=& E\left[ (W^T-U^T) \phi(x) (W^T-U^T) \phi(x^{'}) \right] \\
                &&\rightarrow  \hat W = W - U \\
                &=& E\left[ \hat W^T \phi(x) \hat W^T \phi(x^{'}) \right] \\
                &=& E\left[ \phi(x)^T \hat W \hat W^T \phi(x^{'}) \right] \\
                &=& \phi(x)^T E[\hat W \hat W^T] \phi(x^{'}) \\
                &=& \phi(x)^T S \phi(x^{'})
    \end{array}
    $$
    The above prove use the fact that $w_i$ is independent (iid).
  - *RBF kernel* (stationary kernel): can be derived with the above form
    $$f(x) = \sum_{i=1}^Jw_i\phi_i(x)，w_i \sim \mathcal N\left (0, \frac{\sigma^2}{J} \right), \phi_i(x) = \exp \left(-\frac{(x-c_i)^2}{2l^2} \right)$$
    push the dimension to infinity, we get:
    $$k(x,x^{'}) = \sigma^2 \exp \left( -\frac{(x-x^{'})^2}{2l^2} \right)$$
  - *Neural Network Kernel* is non-stationary kernel:
    $$f(x) = b + \sum_{i=1}^Jv_ih(x;u_i), b\sim\mathcal N(0,\sigma_b^2), v\sim\mathcal N(0,\sigma_v^2/J), u\sim \mathcal N(0,\Sigma).$$
    We get:
    $$k(x, x^{'}) = \frac{2}{\pi} \sin\left( \frac{2\bar x^{\top} \Sigma \bar x^{'}}{\sqrt{(1+2\bar x^{\top} \Sigma \bar x)(1+2\bar x^{'\top} \Sigma \bar x^{'})}} \right)$$
- Inference:
  - With the prior, we get $y|f,x \sim \mathcal N \left(0, k(X,X)+\sigma^2I \right)$ for $y=f(x)+\epsilon$, $f(x) \sim \mathcal{GP}, \epsilon \sim \mathcal N(0, \sigma^2)$. So:
    $$p(y|f,x) = \frac{1}{(2\pi)^{n/2}|k(X,X)+\sigma^2I|^{1/2}} \exp \left(-\frac{1}{2}y^T(k(X,X)+\sigma^2I)^{-1}y \right)$$
    Take log on it, we get:
    $$\log p(y|f,x) = -\frac{1}{2}(k(X,X)+\sigma^2I)^{-1}y - \frac{1}{2}|k(X,X)+\sigma^2I| - \frac{n}{2}\log2\pi$$

## Chapter 19 : Hyperparameter Optimization
- *Configuration Space*: space where hyperparameter choosn from. We can use loguniform to sample hyperparameters which take diverse values.
- Method 1: *Random Search*, most common and most used one. Remenber to run trials multiple times, get the mean the deviation then compare different configurations.
- Synchronous vs. Asynchronous, latter one will not wait for the previous trial result, which is good for trials have different wall times (max running time). And it will drop out bad configuration from time to time.
- *Multi-fidelity* HPO allocates more resources to promising configurations and stop evaluations of poorly performing ones early.
  - *Successive Halving*: start with $N$ configurations, choose a halving constant $\eta$, run $r_{min}$ epoches, keep top $1 / \eta$ fraction, then run $r_{min}\eta$ epoches, keep...
  - *Asynchronous Successive Halving*: don't wait for same run to finish, every time we finish a trial, compare it with the loss in the same run (epoch) condition, if these loss number is less than $\eta$, do more trial for the same epoch, it loss number is larger than $\eta$, check whether it is top $1/\eta$ one, if it is, pormote to next level (more epoch).

## Chapter 20 : Generative Adversarial Networks
<figure style="text-align: center;">
  <img src="https://d2l.ai/_images/gan.svg" alt="GAN" style="background-color: white; display: inline-block;"/>
  <figcaption> GAN </figcaption>
</figure>

- The discriminator is a binary classifier which outputs a saclar prediction $o$, so $D(x) = 1/(1+e^{-o})$, assume the label for $y$ for true data is 1, and 0 for fake data, we have:
  $$\min_D\left\{ -y\log D(x)-(1-y)\log (1-D(x)) \right\}$$
  The generator is $x^{'} = G(z)$, where $z \sim \mathcal N(0,1)$ is called latent variable. The goal is find $G$ that maximize the loss when $y=0$:
  $$\max_G \left \{ -(1-y)\log (1-D(G(z))) \right \} = \max_G \left \{ -\log (1-D(G(z))) \right \}$$
  But in this way if the generator is good, the gradient will be too small for discriminator to evolve, so we choos to minimize this (given $y=1$):
  $$\min_G \left \{ -y\log D(G(z)) \right \}=\min_G \left \{ -\log D(G(z)) \right \}$$
  So the overall objective function is:
  $$\min_D\max_G \left \{ -E_{x\sim \text{Data}}\log D(x) - E_{z \sim \text{Noise}}\log (1-D(G(z)))\right\}$$
- Deep Convolutional GAN
  - Generator use transposed convolution, discriminator use convolution, and the learning rate of them is same, adam smooth term $\beta$ set to 0.5 to deal with rapidly changing gradient. A leakly RELU is used.

## Chapter 21 : Recommender System
- Dataset:
  - *seq-aware* : use the recent data to predict, history data for train.
- *Matrix Factorization*:
  - Let $\mathbf R \in \mathbb R^{m\times n}$ denote the interaction matrix with $m$ users and $n$ items. *Latent Matrix* $\mathbf P \in \mathbb R^{m\times k}$ and $\mathbf Q \in \mathbb R^{n\times k}$, $k \ll m,n$ is the latent factor size. Each row in $\mathbf P$ may represent a user's interest in these latent features, each row in $\mathbf Q$ represent an item's value in these latent features. The predicted ratings are $\hat{\mathbf R} = \mathbf P \mathbf Q^{\top}$. But this way we can not introduce bias, so the predicted one becomes $\hat{\mathbf R_{ui}} = \mathbf p_u \mathbf q_i^{\top} + b_u + b_i$. Then the objective function is:
    $${\arg\min}_{\mathbf{P}, \mathbf{Q}, b} \sum_{(u,i) \in \mathcal{K}} \lVert \mathbf{R}_{ui} - \hat{\mathbf{R}}_{ui} \rVert^2 + \lambda \left( \| \mathbf P \|_F^2 + \| \mathbf Q \|_F^2 + b^2_u + b_i^2 \right)$$
- *AutoRec*:
  - Let $\mathbf R_{*i}$ denote the $i^{th}$ column of the ration matrix, the neural arch is:
    $$h(\mathbf R_{*i}) = f \left ( \mathbf W \cdot g(\mathbf{VR_{*i}}+\mu) + b \right)$$
    Then the objective function is:
    $${\arg\min}_{\mathbf{W}, \mathbf{V}, \mu, b} \sum_{i=1}^M \lVert \mathbf{R}_{*i} - \hat{\mathbf{R}}_{*i} \rVert^2_{\mathcal O} + \lambda \left( \| \mathbf W \|_F^2 + \| \mathbf V \|_F^2 \right)$$
    Where $\|\|_{\mathcal O}$ means only the contribution of observed ratings are considered.
- Type of learning to rank:
    - *Pointwise* approache: MF and AutoRec are both this type, at one time only one individual preference are trained, the order is ignored.
    - *Pairwise* approache: input is pair, we need to select which one is better (consider the order).
    - *Listwise* approache: consider the entire list.
- *Bayeisan Personalized Ranking Loss*:
  - Traning data consist of both positive and negitive pairs (missing values), $(u,i,j)$ represents user $u$ prefer item $i$ over $j$. BPR aims to maximize:
    $$p(\Theta \mid >_u) \propto p(>_u \mid \Theta)p(\Theta)$$
    Where $\Theta$ represents the parameter, $>_u$ represents the desired personalized total ranking of all items for user $u$. We can formulate the maximum posterior estimator (MAE):
    $$
    \begin{array}{lll}
    \text{BPR-OPT} &:=& \ln p(\Theta \mid >_u) \\
                   &\propto& \ln p(>_u \mid \Theta)p(\Theta) \\
                   &=& \ln \prod_{(u,i,j) \in D} \sigma(\hat{y}_{ui} - \hat{y}_{uj}) p(\Theta) \\
                   &=& \sum_{(u,i,j) \in D} \ln \sigma(\hat{y}_{ui} - \hat{y}_{uj}) + \ln p(\Theta) \\
                   &=& \sum_{(u,i,j) \in D} \ln \sigma(\hat{y}_{ui} - \hat{y}_{uj}) - \lambda_\Theta \lVert \Theta \rVert^2
    \end{array}
    $$
    Where we assume $\Theta \sim \mathcal N(0, \lambda_{\Theta} I)$, and $i$ is positive item, $j$ is negitive and missing item.
    This graph shows details:
    <figure style="text-align: center;">
      <img src="https://d2l.ai/_images/rec-ranking.svg" alt="rec-ranking" style="background-color: white; display: inline-block;"/>
      <figcaption> rec-ranking </figcaption>
    </figure>
- *Hinge Loss*: $\sum_{(u,i,j) \in D} \max (m-\hat{y}_{ui}+\hat{y}_{uj}, 0)$ often used in SVM, where $m$ is the safety margin size.
- *Neural Collaborative Filtering*: consist of 2 subnets.
  - *GMF* subnet: $x = p_u \odot q_i$, $\hat y_{ui} = \alpha(h^{\top}x)$, $p$ and $q$ are latent matrix row, $\hat y_{ui}$ is user $u$ score on item $i$.
  - *MLP* subnet: its input is concatenation of user and item embeddings.
    $$
    \begin{align*}
    \mathbf{z}^{(1)} &= \phi_1(\mathbf{U}_u, \mathbf{V}_i) = [\mathbf{U}_u, \mathbf{V}_i] \\
    \phi^{(2)}(\mathbf{z}^{(1)}) &= \alpha^{1}(\mathbf{W}^{(2)} \mathbf{z}^{(1)} + \mathbf{b}^{(2)}) \\
    &\vdots \\
    \phi^{(L)}(\mathbf{z}^{(L-1)}) &= \alpha^{L}(\mathbf{W}^{(L)} \mathbf{z}^{(L-1)} + \mathbf{b}^{(L)}) \\
    \hat{y}_{ui} &= \alpha(\mathbf{h}^\top \phi^{L}(\mathbf{z}^{(L-1)}))
    \end{align*}
    $$
  - To combine them, we take the last layer and concatenate: $\hat y_{ui} = \sigma \left ( h^{\top}[x, \phi^L(z^{L-1})] \right )$.
  - To evaluate the result, we use a hit rate at given cutting off $\mathscr l$: $\frac{1}{m}\sum_{u\in \mathcal U} \mathbf 1(\text{rank}_{u,g_u} \leq \mathscr l)$ and area under the ROC curve (AUC):
    $$ \text{AUC} = \frac 1 m \sum_{u\in \mathcal U} \frac{1}{|\mathcal I \setminus S_u|} \sum_{j\in \mathcal I \setminus S_u} \mathbf 1(\text{rank}_{u,g_u} < \text{rank}_{u,j})$$
    Where $\mathcal I$ is the item set, $S_u$ is the candidate items of user $u$.
- *Caser*: The goal of Caser is to recommend item by considering user general tastes as well as short-term intention.
  - Let $S^u=(S^u_1, \ldots, S^u_{|S_u|})$ denotes the ordered sequence, consider $L$ recent items:
    $$E^{(u,t)} = \left [ qS^u_{t-L}, \ldots, qS^u_{t-2}, S^u_{t-1} \right ]^{\top}$$
    where $Q\in \mathbb R^{n\times k}$ represents items embeddings, $E^{(u,t)} \in \mathbb R^{L\times k}$.
    <figure style="text-align: center;">
      <img src="https://d2l.ai/_images/rec-caser.svg" alt="rec-caser" style="background-color: white; display: inline-block;"/>
      <figcaption> rec-caser </figcaption>
    </figure>
    Left is horizontal convolutional layer, right is vertical. $p_u$ and $v_i$ (another one for item) are user and item embedding. So basically, we input the $E$ input convolution layer get feature $z$, concatenates with user embedding $p_u$, get feature $f$, then with item embedding $v_i^{pos}$ and $v_i^{neg}$ to get $\hat y^{pos} = f^{\top}v_i^{pos} + b^{pos}, \hat y^{neg} = f^{\top}v_i^{neg} + b^{neg}$, get the BPR loss.
- Feature-Rich Recommender System:
  - *Click-through rate*: click / impression (show times) * 100%.
  - *Factorizatin Machine*: given $x \in \mathbb R^d$ with $d$ fields, feature embedding $V \in \mathbb R^{d\times k}$
    $$ \hat{y}(x) = \mathbf{w}_0 + \sum_{i=1}^d \mathbf{w}_i x_i + \sum_{i=1}^d\sum_{j=i+1}^d \langle\mathbf{v}_i, \mathbf{v}_j\rangle x_i x_j $$
    Then reformulate the equation to decrease the complexity:
    $$
    \begin{split}\begin{aligned}
    &\sum_{i=1}^d \sum_{j=i+1}^d \langle\mathbf{v}_i, \mathbf{v}_j\rangle x_i x_j \\
     &= \frac{1}{2} \sum_{i=1}^d \sum_{j=1}^d\langle\mathbf{v}_i, \mathbf{v}_j\rangle x_i x_j - \frac{1}{2}\sum_{i=1}^d \langle\mathbf{v}_i, \mathbf{v}_i\rangle x_i x_i \\
     &= \frac{1}{2} \big (\sum_{i=1}^d \sum_{j=1}^d \sum_{l=1}^k\mathbf{v}_{i, l} \mathbf{v}_{j, l} x_i x_j - \sum_{i=1}^d \sum_{l=1}^k \mathbf{v}_{i, l} \mathbf{v}_{i, l} x_i x_i \big)\\
     &=  \frac{1}{2} \sum_{l=1}^k \big ((\sum_{i=1}^d \mathbf{v}_{i, l} x_i) (\sum_{j=1}^d \mathbf{v}_{j, l}x_j) - \sum_{i=1}^d \mathbf{v}_{i, l}^2 x_i^2 \big ) \\
     &= \frac{1}{2} \sum_{l=1}^k \big ((\sum_{i=1}^d \mathbf{v}_{i, l} x_i)^2 - \sum_{i=1}^d \mathbf{v}_{i, l}^2 x_i^2)
     \end{aligned}\end{split}
    $$
  - *Deep FM*: $\hat{y} = \sigma(\hat{y}^{(FM)} + \hat{y}^{(DNN)})$
    <figure style="text-align: center;">
      <img src="https://d2l.ai/_images/rec-deepfm.svg" alt="rec-deepfm" style="background-color: white; display: inline-block;"/>
      <figcaption> rec-deepfm </figcaption>
    </figure>
- FAQ:
  - What is the field? Q: for example, we have 100 samples, total shape is [100, 500], so it has 500 fields, then we put it into embedding layer get [100, 500, embedding_dims].
<!-- <figure style="text-align: center;">
  <img src="https://d2l.ai/_images/gan.svg" alt="GAN" style="background-color: white; display: inline-block;"/>
  <figcaption> GAN </figcaption>
</figure> -->

<figure style="text-align: center;">
  <img src="https://d2l.ai/_images/gan.svg" alt="GAN" style="background-color: white; display: inline-block;"/>
  <figcaption> GAN </figcaption>
</figure>

<!-- <img alt="neural-style" src="https://d2l.ai/_images/neural-style.svg" style="background-color: white; display: inline-block;"> -->

在**深度学习（DL）工程**和**硬件优化**方面，需要掌握一系列工具、技术和最佳实践，以确保模型能够高效训练、优化和部署。  

---

# **1. 深度学习工程**
**目标**：不仅要训练模型，还要能够在实际应用中高效地**数据处理、训练、调优、部署和维护**。

## **1.1 数据工程**
深度学习的性能很大程度上依赖于数据质量和预处理效率。

### **(1) 数据收集与存储**
- **结构化数据**（SQL, Pandas, BigQuery）
- **图像数据**（OpenCV, PIL, TensorFlow Datasets）
- **文本数据**（NLTK, Hugging Face Datasets）
- **流数据**（Kafka, Apache Spark）

### **(2) 数据预处理**
- **标准化 / 归一化**（Min-Max Scaling, Z-score）
- **数据增强**（图像：旋转、裁剪；文本：同义词替换）
- **降维**（PCA, t-SNE, UMAP）
- **缺失值处理**（均值填充、插值）

### **(3) 数据加载优化**
- **批量加载（Batch Loading）**
- **多线程 / 多进程数据预处理（Dataloader, TensorFlow tf.data）**
- **TFRecord / HDF5**（二进制格式加速数据读取）

---

## **1.2 训练与超参数调优**
深度学习模型训练是一个计算密集型过程，需要高效的**优化策略**和**超参数调整**。

### **(1) 训练优化**
- **优化器选择**
  - SGD（标准梯度下降）
  - Adam / RMSprop（自适应优化）
  - LARS / LAMB（用于大规模分布式训练）
  
- **正则化**
  - Dropout（随机丢弃神经元）
  - Batch Normalization（批量归一化）
  - Weight Decay（L2 正则化）

- **梯度裁剪（Gradient Clipping）**
  - 解决梯度爆炸问题

### **(2) 超参数优化**
自动搜索最优超参数（例如学习率、batch size、权重初始化）。
- **Grid Search（网格搜索）**
- **Random Search（随机搜索）**
- **Bayesian Optimization（贝叶斯优化）**
- **Hyperband（高效采样）**
- **Optuna / Ray Tune（自动化超参数调优工具）**

---

## **1.3 训练加速**
大规模训练时需要高效的训练加速技术：

### **(1) GPU 加速**
- 训练时尽可能利用 **CUDA** / **cuDNN**
- **混合精度训练（Mixed Precision）**：使用 FP16（Half Precision）加速计算
- **数据并行（DataParallel）** vs. **模型并行（ModelParallel）**
  
### **(2) 分布式训练**
- **单机多卡（Multi-GPU Training）**
  - PyTorch `DataParallel`
  - PyTorch `DistributedDataParallel (DDP)`
  
- **多机多卡（Multi-Node Training）**
  - TensorFlow `MirroredStrategy`
  - Horovod（Uber 提出的高效分布式训练框架）

---

## **1.4 部署与推理优化**
深度学习不仅要训练，还要在**边缘设备**或**服务器端**高效推理。

### **(1) 模型压缩**
- **剪枝（Pruning）**：去掉不重要的权重
- **量化（Quantization）**：
  - **8-bit INT 量化**（TensorRT, TFLite）
  - **混合精度推理（FP16, INT8）**

- **知识蒸馏（Knowledge Distillation）**：
  - 用大模型训练小模型，提高推理效率

### **(2) 推理框架**
- **ONNX（Open Neural Network Exchange）**：模型通用格式，可用于 PyTorch / TensorFlow 互转
- **TensorRT（NVIDIA）**：高效的 GPU 加速推理
- **TVM（Apache）**：自动优化模型推理

### **(3) 部署方式**
- **服务器部署**
  - Flask / FastAPI（REST API 部署）
  - TensorFlow Serving / TorchServe（高效模型服务）

- **移动端 / 边缘部署**
  - TensorFlow Lite（TFLite）
  - CoreML（iOS 设备）
  - NVIDIA Jetson（嵌入式 AI）

---

# **2. 硬件优化**
深度学习的计算量极大，硬件的优化能**显著提高训练和推理速度**。

## **2.1 GPU 计算**
GPU 是深度学习的核心计算设备，NVIDIA CUDA 生态至关重要。

### **(1) GPU 编程基础**
- CUDA 编程（掌握 Kernel 编写）
- cuDNN（深度学习优化库）
- Tensor Core（用于混合精度计算）
  
### **(2) GPU 训练优化**
- **减少 CPU-GPU 传输**（优化 `pin_memory=True`）
- **梯度累积（Gradient Accumulation）**，减少显存占用
- **使用 FP16 训练**（提高吞吐量）

---

## **2.2 分布式计算**
适用于**超大规模数据训练**（如 GPT、Llama 等模型）。

### **(1) 并行策略**
- **数据并行（Data Parallelism）**
  - 复制模型到多个 GPU，每个 GPU 训练不同数据
  - PyTorch `DistributedDataParallel (DDP)`

- **模型并行（Model Parallelism）**
  - 适用于超大模型（如 GPT-4）
  - DeepSpeed / Megatron-LM 优化

- **流水线并行（Pipeline Parallelism）**
  - 将不同层分配到不同 GPU，提高计算效率
  - **适用于 Transformer 训练**

### **(2) 高效通信**
- **NCCL（NVIDIA Collective Communication Library）**：优化 GPU 之间的通信
- **RDMA（远程直接内存访问）**：用于 GPU 服务器间高速通信

---

## **2.3 专用 AI 硬件**
除了 GPU，AI 训练还可以用专用芯片加速：
- **TPU（Google）**：专门优化深度学习计算
- **Graphcore IPU**（稀疏计算优化）
- **Cerebras Wafer-Scale Engine**（超大规模 AI 计算）

---

# **3. 总结**
| **类别** | **关键内容** |
|----------|--------------|
| **数据工程** | 数据清洗、数据增强、数据加载优化 |
| **训练优化** | 超参数调优、正则化、优化器选择 |
| **训练加速** | GPU 加速、混合精度、分布式训练 |
| **部署优化** | 模型量化、剪枝、TensorRT 加速 |
| **硬件优化** | CUDA、NCCL、TPU/FPGA |

你已经有**矩阵分解和 Rust 经验**，如果想深入工程优化，可以：
1. **研究 PyTorch DDP / DeepSpeed**（分布式训练优化）
2. **学习 CUDA / cuDNN 编程**（低级 GPU 加速）
3. **尝试 TensorRT / ONNX**（推理加速）

这将让你在 **深度学习工程 & 硬件优化** 方面具备更强的竞争力 🚀

## 梯度检查点

梯度检查点（Gradient Checkpointing，也叫 Checkpointing 或 Recomputation）是一种在训练深度神经网络时用来**节省内存**的技术，尤其在网络非常深或批量大小（batch size）较大时非常有用。它的核心思想是：在内存有限的情况下，通过牺牲一些计算时间来减少存储激活值（中间输出）的内存需求。下面我将详细解释它的原理、实现方式以及优缺点。

---

### 背景：为什么需要梯度检查点？
在深度神经网络的训练中，反向传播需要用到前向传播时每一层的激活值。这些激活值通常会被存储在内存中，以便在计算梯度时直接使用。然而：
- 对于深层网络（比如 ResNet-101、Transformer 等），层数非常多，激活值的内存占用会变得非常大。
- 当批量大小增加时，激活值的存储需求进一步线性增长。
- 如果显存（如 GPU 内存）不足，可能无法训练模型，或者只能使用很小的批量大小，影响训练效果。

梯度检查点的目标是通过**不存储所有激活值**，而是在需要时重新计算它们，从而大幅减少内存占用。

---

### 基本原理
正常情况下，训练一个神经网络的过程是：
1. **前向传播**：从输入到输出，计算每一层的激活值并存储。
2. **反向传播**：利用存储的激活值和损失函数的梯度，计算每一层参数的梯度。

梯度检查点的工作方式是：
- 在前向传播中，**只保存部分关键层的激活值**（这些点称为“检查点”），而不是每一层的激活值。
- 在反向传播时，对于未保存激活值的层，通过从最近的检查点重新运行前向传播来**重新计算激活值**，然后继续计算梯度。

这种方法用额外的计算（重新计算激活值）换取了内存的节省。

---

### 详细步骤
假设一个简单的网络有 5 层：\( L_1 \rightarrow L_2 \rightarrow L_3 \rightarrow L_4 \rightarrow L_5 \)。

#### 普通训练
1. 前向传播：计算并存储所有层的激活值 \( A_1, A_2, A_3, A_4, A_5 \)。
2. 反向传播：从 \( L_5 \) 开始，利用 \( A_5, A_4, \ldots, A_1 \) 计算每一层的梯度。
3. 内存需求：存储所有 \( A_1 \) 到 \( A_5 \)，假设每层激活值占 \( M \) 个字节，总共 \( 5M \)。

#### 使用梯度检查点
假设我们选择 \( L_2 \) 和 \( L_4 \) 作为检查点：
1. **前向传播**：
   - 计算 \( L_1 \rightarrow L_5 \)，但只保存检查点的激活值 \( A_2 \) 和 \( A_4 \)。
   - 其他层的激活值 \( A_1, A_3, A_5 \) 在前向传播后被丢弃。
   - 内存需求：仅存储 \( A_2 \) 和 \( A_4 \)，即 \( 2M \)。

2. **反向传播**：
   - 从 \( L_5 \) 开始计算梯度，需要 \( A_4 \)（已保存）和 \( A_5 \)（未保存）。
   - 从 \( A_4 \) 重新运行前向传播 \( L_4 \rightarrow L_5 \)，重新计算 \( A_5 \)，然后计算 \( L_5 \) 和 \( L_4 \) 的梯度。
   - 继续到 \( L_3 \)，需要 \( A_2 \)（已保存）和 \( A_3 \)（未保存）。
   - 从 \( A_2 \) 重新运行 \( L_2 \rightarrow L_3 \)，计算 \( A_3 \)，然后计算 \( L_3 \) 的梯度。
   - 最后从 \( A_2 \) 和输入重新计算 \( A_1 \)，完成 \( L_2 \) 和 \( L_1 \) 的梯度。

3. **内存节省**：只存 2 个检查点的激活值（\( 2M \)），而不是 5 个（\( 5M \)）。
4. **额外计算**：需要重新运行部分前向传播（例如 \( L_2 \rightarrow L_3 \) 和 \( L_4 \rightarrow L_5 \)）。

---

### 检查点的选择
如何选择检查点是一个关键问题：
- **均匀分布**：例如每隔 \( k \) 层设置一个检查点（如每 10 层）。
- **动态选择**：根据内存需求和计算复杂度，优先选择内存占用大的层作为检查点。
- **理论最优**：研究表明，最优检查点策略可以将内存需求从 \( O(N) \)（N 为层数）降低到 \( O(\sqrt{N}) \)，但实现起来较复杂。

现代深度学习框架（如 PyTorch、TensorFlow）通常内置了检查点功能，用户只需指定哪些层或子模块作为检查点。

---

### 数学分析
假设网络有 \( N \) 层，每层激活值占 \( M \) 字节：
- **普通训练**：内存需求 \( N \times M \)。
- **检查点训练**：
  - 设检查点数为 \( K \)（\( K < N \)），内存需求为 \( K \times M \)。
  - 额外计算量与 \( N/K \) 成正比（每个检查点负责的层数）。
  - 最优情况下，\( K \approx \sqrt{N} \)，内存降为 \( O(\sqrt{N} \times M) \)。

例如，\( N = 100 \)：
- 普通训练：\( 100M \)。
- 检查点训练（\( K = 10 \)）：\( 10M \) + 少量额外计算。

---

### 优点
1. **内存效率**：显著减少激活值的存储需求，允许训练更深的网络或使用更大的批量大小。
2. **灵活性**：在显存受限的设备上也能运行复杂模型。
3. **广泛适用**：适用于 CNN、RNN、Transformer 等各种架构。

---

### 缺点
1. **计算开销**：重新计算激活值增加了训练时间，通常比普通训练慢 20%-50%，具体取决于检查点数量和网络结构。
2. **实现复杂性**：需要手动指定检查点或依赖框架支持，调试可能更困难。
3. **不适合所有场景**：如果内存不是瓶颈，检查点反而会降低效率。

---

### 实际应用
- **PyTorch 示例**：
  ```python
  import torch
  import torch.nn as nn
  from torch.utils.checkpoint import checkpoint

  class MyModel(nn.Module):
      def __init__(self):
          super().__init__()
          self.layer1 = nn.Linear(1024, 1024)
          self.layer2 = nn.Linear(1024, 1024)
          self.layer3 = nn.Linear(1024, 1024)

      def forward(self, x):
          x = checkpoint(self.layer1, x)  # 检查点包裹层1
          x = checkpoint(self.layerThe layer2(x)      # 普通层
          x = checkpoint(self.layer3, x)  # 检查点包裹层3
          return x

  model = MyModel()
  input = torch.randn(32, 1024)  # batch_size = 32
  output = model(input)
  ```

- **大模型**：如 GPT、LLaMA 或大型 Transformer，层数可能高达数百层，使用梯度检查点可以显著降低显存需求。

---

### 总结
梯度检查点是一种以时间换空间的优化技术，通过减少激活值的存储来降低内存需求，非常适合显存受限或深层网络的场景。它的核心是选择部分检查点并在反向传播时重新计算其他层的激活值。虽然增加了计算开销，但在内存是瓶颈时，它能让“不可能的任务”变得可行。

如果你对实现细节或具体场景有疑问，随时告诉我，我可以进一步展开！